# developing external testting tricks?

## the OpenAI/BigCode variant only works on UNix

## I know that tempdir and subprocess works and does timeout... but can you actually recover from that?
should I allow `wgpu-shadertoy` to take a code arg in cli? instead of just
this will only work for singlepass shaders just now.

we hope that wgpu 22 and get shader compilation info gets upstreamed to wgpu-py soon (and I will participate).

In [8]:
import os
import tempfile
import subprocess


file_template = """
from wgpu_shadertoy import Shadertoy

shader_code = '''{}'''

shader = Shadertoy(shader_code, shader_type="glsl", offscreen=True)

if __name__ == "__main__":
    shader.show()
    shader.snapshot(0.0)
"""

def run_shader_in_subprocess(shader_code, timeout=5):
    status = "ok" # default case
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False, encoding="utf-8") as f:
        f.write(file_template.format(shader_code))
        f.flush()
        try:
            p = subprocess.run(["python", f.name], capture_output=True, timeout=timeout)
            
        except subprocess.SubprocessError as e:
            if isinstance(e, subprocess.TimeoutExpired):
                status = "timeout"
            else:
                status = "error"

        if p.stderr != b"":
            status = "error"
    
    # cleanup temp file
    os.remove(f.name)

    return status


In [9]:
new_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    // Time varying pixel color
    vec3 col = 0.5 + 0.5*cos(iTime+uv.xyx+vec3(0,2,4));

    // Output to screen
    fragColor = vec4(col,1.0);
}

"""

# this panics because it loses device
minimal_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord ) {

    vec3 col = vec3(0.0);
    float incr = 0.1;
    for (float i = 0.5; i < 3.0; i += max(0.0, iTime)) {
        col += vec3(0.2);
        // continue;
    }
    fragColor = vec4(col, 1.0);
}

"""

In [10]:
print(run_shader_in_subprocess(new_code))
print(run_shader_in_subprocess(minimal_code, 10))

C:\Users\Jan\AppData\Local\Temp\tmpiyvbdxww.py
ok
C:\Users\Jan\AppData\Local\Temp\tmplrlcdzqp.py
error


In [4]:
from annotate import run_shader

print(run_shader(new_code))
print(run_shader(minimal_code))

ok
ok
